# 🔧 Chat2DB - Advanced Usage

This notebook covers advanced Chat2DB features:

1. Testing the Letta/Athena connection
2. Using different LLM providers
3. Custom database connections
4. Conversation memory and context
5. Safety features and guardrails
6. Data analysis workflows

**🦉 Athena** is your data & analytics oracle for all SQL generation and explanations.

---

## 1️⃣ Test Letta/Athena Connection

Verify your Letta AI agent (Athena) is accessible.

In [ ]:
from letta_chat2db import test_letta_connection

# Test the connection to Letta and Athena
test_letta_connection()

In [ ]:
# Test sending a message to Athena directly
from letta_chat2db import LettaChat2DB

letta = LettaChat2DB()
response = letta.send_message("Hello Athena! Can you help me with SQL queries?")
print(f"🦉 Athena says: {response.content}")

## 2️⃣ Using Different LLM Providers

Chat2DB uses Letta as the AI backend. Configure the LLM provider in Letta's settings.

Supported providers via Letta:
- **OpenAI** (GPT-4, GPT-4o)
- **OpenRouter** (Claude, Llama, Mistral, etc.)
- **Ollama** (local models)
- **Z.ai** and other OpenAI-compatible APIs

In [ ]:
# Chat2DB uses Athena via Letta
# Configure the LLM in your Letta server settings or .env file
from chat2db import Chat2DB

chat = Chat2DB(connection_code="sample-northwind-sqlite")

# Athena uses whatever LLM is configured in Letta
result = chat.ask("What tables are in this database?")
result

## 3️⃣ Parse ReportBurster Connections

Directly work with ReportBurster's XML connection files.

In [ ]:
from rb_connections import ReportBursterConnections, print_connections

# Print all available connections
print_connections()

In [ ]:
# Get connection details programmatically
manager = ReportBursterConnections()
connections = manager.list_connections()

for conn in connections:
    print(f"Connection: {conn.code}")
    print(f"  Type: {conn.db_type}")
    print(f"  Driver: {conn.driver}")
    print(f"  URL: {conn.url}")
    print()

## 4️⃣ Manual JDBC Connection

Connect directly via JDBC if you need more control.

In [ ]:
from rb_connections import ReportBursterConnections
import pandas as pd

# Get the connection manager
manager = ReportBursterConnections()

# Connect to a specific database
conn = manager.connect("sample-northwind-sqlite")

# Execute queries manually
cursor = conn.cursor()
cursor.execute("SELECT * FROM customers LIMIT 5")

# Get results
columns = [desc[0] for desc in cursor.description]
rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=columns)
df

In [ ]:
# Close the connection
manager.close()

## 5️⃣ Custom Database Connection

Connect to a database not configured in ReportBurster.

In [ ]:
from rb_connections import DatabaseConnection
from chat2db import Chat2DB

# Create a custom connection config
custom_conn = DatabaseConnection(
    code="custom-postgres",
    name="My Custom PostgreSQL",
    db_type="postgresql",
    host="host.docker.internal",  # Use this for local DBs from Docker
    port="5432",
    database="mydb",
    userid="postgres",
    userpassword="password"
)

# Connect using the custom config
# chat = Chat2DB(connection_config=custom_conn)
# chat.ask("Show all tables")

## 6️⃣ Safety Features

Chat2DB includes guardrails to prevent dangerous operations like DELETE, DROP, UPDATE.

In [ ]:
from chat2db import Chat2DB

# Default: dangerous SQL is blocked
chat = Chat2DB(connection_code="sample-northwind-sqlite")

# This will be blocked:
result = chat.ask("Delete all records from customers")
print(f"Error: {result.error}")  # Shows blocking message

In [ ]:
# To disable safety (not recommended for production!):
# chat = Chat2DB(
#     connection_code="sample-northwind-sqlite",
#     block_dangerous=False
# )

## 7️⃣ Conversation Memory

Athena maintains context across questions using Letta's memory system.

In [ ]:
chat = Chat2DB(connection_code="sample-northwind-sqlite")

# First question establishes context
result1 = chat.ask("What were total sales in 1997?")
print(f"Result: {result1.df}")

# Follow-up uses context (Athena remembers the conversation)
result2 = chat.ask("Now break that down by category")
print(f"Result: {result2.df}")

## 8️⃣ Data Analysis Workflow

Complete example: Explore → Analyze → Visualize

In [ ]:
from chat2db import Chat2DB

chat = Chat2DB(connection_code="sample-northwind-sqlite")

# Step 1: Explore the data
print("📊 Database Schema:")
print(chat.schema())

In [ ]:
# Step 2: Ask analytical questions
result = chat.ask("What are the top 10 products by total revenue?")
result.df

In [ ]:
# Step 3: Create visualizations
import plotly.express as px

df = result.df

if len(df) > 0 and len(df.columns) >= 2:
    # Assume first column is name, second is value
    fig = px.bar(
        df, 
        x=df.columns[0], 
        y=df.columns[1],
        title="Top Products by Revenue"
    )
    fig.show()

In [ ]:
# Step 4: Deeper analysis
result = chat.ask("Show monthly sales trend for 1997")
df = result.df

if len(df) > 0:
    fig = px.line(
        df, 
        x=df.columns[0], 
        y=df.columns[1],
        title="Monthly Sales Trend",
        markers=True
    )
    fig.show()

## 9️⃣ Export Results

Save your query results for reporting.

In [ ]:
# Export to CSV
result = chat.ask("List all customers with their total orders")
result.df.to_csv("/app/notebooks/customers_report.csv", index=False)
print("✅ Exported to customers_report.csv")

In [ ]:
# Export to Excel
result.df.to_excel("/app/notebooks/customers_report.xlsx", index=False)
print("✅ Exported to customers_report.xlsx")

## 🔟 Clean Up

In [ ]:
chat.close()

---

## 📚 Reference

### Chat2DB Methods

| Method | Description |
|--------|-------------|
| `list_connections()` | List available ReportBurster connections |
| `connect(code)` | Connect to a database |
| `schema()` | Get current database schema |
| `ask(question)` | Ask Athena a natural language question |
| `sql(query)` | Execute raw SQL |
| `interactive()` | Launch chat widget |
| `close()` | Close all connections |

### QueryResult Properties

| Property | Description |
|----------|-------------|
| `df` | Pandas DataFrame with results |
| `sql` | Generated SQL query |
| `explanation` | Athena's explanation of results |
| `row_count` | Number of rows returned |
| `execution_time_ms` | Query execution time |
| `error` | Error message if query failed |

### Environment Variables

| Variable | Description |
|----------|-------------|
| `LETTA_BASE_URL` | Letta server URL (default: http://localhost:8283) |
| `AGENT_ATHENA_ID` | Athena agent ID from Letta |

# 🔧 Chat2DB - Advanced Usage

This notebook covers advanced Chat2DB features:

1. **Multi-Agent Workflows** - Work with specialized AI agents
2. Using different LLM providers
3. Custom database connections
4. Conversation memory and context
5. Safety features and guardrails
6. Data analysis workflows

---

## 1️⃣ Multi-Agent Workflows

Chat2DB supports multiple AI agents, each with different specializations. Chain them together for powerful workflows!

In [ ]:
from chat2db import Chat2DB, Agents

chat = Chat2DB(connection_code="sample-northwind-sqlite")

# List all configured agents
chat.list_agents()

### Workflow: Explore → Analyze → Quality Check

Use different agents for different stages of analysis:

In [ ]:
# Step 1: Ask Athena for general data analysis
result_athena = chat.athena().ask("What are the top 10 customers by total order value?")
print(f"🦉 Athena found {result_athena.row_count} top customers")
result_athena.df

In [ ]:
# Step 2: Ask Apollo for visualization recommendations
result_apollo = chat.apollo().ask(
    "Based on our top customers data, what's the best way to visualize "
    "customer value distribution? Show me orders by month for the top customer."
)
print(f"☀️ Apollo recommends:")
print(result_apollo.explanation)

In [ ]:
# Step 3: Ask Artemis to check for data quality issues
result_artemis = chat.artemis().ask(
    "Are there any customers without orders, "
    "or orders without valid customer references?"
)
print(f"🏹 Artemis data quality check:")
print(result_artemis.explanation)
result_artemis.df

### Fluent API - Chain Methods

The agent methods return `self`, so you can chain them:

In [ ]:
# One-liner: switch agent and ask in a single line
chat.hermes().ask("Show me the relationship between products and categories")

# Chain multiple operations
result = (
    chat
    .connect("sample-northwind-sqlite")
    .athena()
    .ask("Count products per category")
)
result.df

---

## 2️⃣ Using Different LLM Providers

Chat2DB supports multiple LLM backends. Configure in `.env` or at runtime.

In [ ]:
# Option 1: Use Letta AI (default - configured in .env)
from chat2db import Chat2DB
chat = Chat2DB(connection_code="sample-northwind-sqlite")

# The Letta agent is already configured via LETTA_API_BASE_URL and LETTA_AGENT_ID

In [ ]:
# Option 2: Use alternative providers directly
from llm_providers import OpenAIProvider, OpenRouterProvider, OllamaProvider

# Example: Use OpenRouter with Claude
# provider = OpenRouterProvider(model="anthropic/claude-3-haiku")

# Example: Use local Ollama
# provider = OllamaProvider(model="llama3")

# Example: Use OpenAI directly
# provider = OpenAIProvider(model="gpt-4o-mini")

## 2️⃣ Test Letta Connection

Verify your Letta AI agent is accessible.

In [ ]:
from letta_chat2db import test_letta_connection

# Test the connection to Letta
test_letta_connection()

In [ ]:
# List available Letta agents
from letta_chat2db import LettaChat2DB

letta = LettaChat2DB()
agents = letta.list_agents()

print("Available Letta Agents:")
for agent in agents:
    print(f"  - ID: {agent.get('id')}")
    print(f"    Name: {agent.get('name', 'Unnamed')}")
    print()

## 3️⃣ Parse ReportBurster Connections

Directly work with ReportBurster's XML connection files.

In [ ]:
from rb_connections import ReportBursterConnections, print_connections

# Print all available connections
print_connections()

In [ ]:
# Get connection details programmatically
manager = ReportBursterConnections()
connections = manager.list_connections()

for conn in connections:
    print(f"Connection: {conn.code}")
    print(f"  Type: {conn.db_type}")
    print(f"  Driver: {conn.driver}")
    print(f"  URL: {conn.url}")
    print()

## 4️⃣ Manual JDBC Connection

Connect directly via JDBC if you need more control.

In [ ]:
from rb_connections import ReportBursterConnections
import pandas as pd

# Get the connection manager
manager = ReportBursterConnections()

# Connect to a specific database
conn = manager.connect("sample-northwind-sqlite")

# Execute queries manually
cursor = conn.cursor()
cursor.execute("SELECT * FROM customers LIMIT 5")

# Get results
columns = [desc[0] for desc in cursor.description]
rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=columns)
df

In [ ]:
# Close the connection
manager.close()

## 5️⃣ Custom Database Connection

Connect to a database not configured in ReportBurster.

In [ ]:
from rb_connections import DatabaseConnection
from chat2db import Chat2DB

# Create a custom connection config
custom_conn = DatabaseConnection(
    code="custom-postgres",
    name="My Custom PostgreSQL",
    db_type="postgresql",
    host="host.docker.internal",  # Use this for local DBs from Docker
    port="5432",
    database="mydb",
    userid="postgres",
    userpassword="password"
)

# Connect using the custom config
# chat = Chat2DB(connection_config=custom_conn)
# chat.ask("Show all tables")

## 6️⃣ Safety Features

Chat2DB includes guardrails to prevent dangerous operations.

In [ ]:
from chat2db import Chat2DB

# Default: dangerous SQL is blocked
chat = Chat2DB(connection_code="sample-northwind-sqlite")

# This will be blocked:
result = chat.ask("Delete all records from customers")
print(result.error)  # Shows blocking message

In [ ]:
# To disable safety (not recommended for production!):
# chat = Chat2DB(
#     connection_code="sample-northwind-sqlite",
#     block_dangerous=False
# )

## 7️⃣ Conversation Memory

Chat2DB maintains context across questions (when using Letta).

In [ ]:
chat = Chat2DB(connection_code="sample-northwind-sqlite")

# First question establishes context
result1 = chat.ask("What were total sales in 1997?")
print(f"Result: {result1.df}")

# Follow-up uses context
result2 = chat.ask("Now break that down by category")
print(f"Result: {result2.df}")

# Clear history to start fresh
chat.clear_history()

## 8️⃣ Data Analysis Workflow

Complete example: Explore → Analyze → Visualize

In [ ]:
import plotly.express as px
from chat2db import Chat2DB

chat = Chat2DB(connection_code="sample-northwind-sqlite")

# Step 1: Explore the data
print("📊 Database Schema:")
chat.show_schema()

In [ ]:
# Step 2: Ask analytical questions
result = chat.ask("What are the top 10 products by total revenue?")
result.df

In [ ]:
# Step 3: Create visualizations
df = result.df

if len(df) > 0 and len(df.columns) >= 2:
    # Assume first column is name, second is value
    fig = px.bar(
        df, 
        x=df.columns[0], 
        y=df.columns[1],
        title="Top Products by Revenue"
    )
    fig.show()

In [ ]:
# Step 4: Deeper analysis
result = chat.ask("Show monthly sales trend for 1997")
df = result.df

if len(df) > 0:
    fig = px.line(
        df, 
        x=df.columns[0], 
        y=df.columns[1],
        title="Monthly Sales Trend",
        markers=True
    )
    fig.show()

## 9️⃣ Export Results

Save your query results for reporting.

In [ ]:
# Export to CSV
result = chat.ask("List all customers with their total orders")
result.df.to_csv("/app/notebooks/customers_report.csv", index=False)
print("✅ Exported to customers_report.csv")

In [ ]:
# Export to Excel
result.df.to_excel("/app/notebooks/customers_report.xlsx", index=False)
print("✅ Exported to customers_report.xlsx")

## 🔟 Clean Up

In [ ]:
chat.close()

---

## 📚 Reference

### Chat2DB Methods

| Method | Description |
|--------|-------------|
| `list_connections()` | List available ReportBurster connections |
| `connect(code)` | Connect to a database |
| `show_schema()` | Display current database schema |
| `ask(question)` | Ask a natural language question |
| `sql(query)` | Execute raw SQL |
| `interactive()` | Launch chat widget |
| `clear_history()` | Clear conversation memory |
| `close()` | Close all connections |

### QueryResult Properties

| Property | Description |
|----------|-------------|
| `df` | Pandas DataFrame with results |
| `sql` | Generated SQL query |
| `explanation` | AI explanation of results |
| `row_count` | Number of rows returned |
| `execution_time_ms` | Query execution time |
| `error` | Error message if query failed |